In [ ]:
import torch
print(torch.__version__)  # Kiểm tra phiên bản PyTorch
print(torch.cuda.is_available())  # Kiểm tra xem PyTorch có nhận GPU không
print(torch.version.cuda)  # Kiểm tra phiên bản CUDA mà PyTorch đang sử dụng
print(torch.cuda.get_device_name(0))  # Kiểm tra tên GPU


2.5.1+cu121
True
12.1
NVIDIA GeForce RTX 4060 Laptop GPU


In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from tqdm import tqdm

In [ ]:
# Set seeds for reproducibility
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
np.random.seed(42)

# Define dataset paths
dataset_path = './Multi Cancer/Cervical Cancer'
cervix_categories = ['cervix_dyk', 'cervix_koc', 'cervix_mep', 'cervix_pab', 'cervix_sfi']

# Load dataset file paths and labels
filepaths, labels = [], []
for label, category in enumerate(cervix_categories):
    category_path = os.path.join(dataset_path, category)
    for file in os.listdir(category_path):
        filepaths.append(os.path.join(category_path, file))
        labels.append(label)

# Convert to DataFrame
df = pd.DataFrame({'filepath': filepaths, 'label': labels})

# Split dataset into train, validation, and test sets
train_df, temp_df = train_test_split(df, test_size=0.3, stratify=df['label'], random_state=42)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [4]:
# Custom dataset class
class CervicalCancerDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['filepath']
        label = self.df.iloc[idx]['label']
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)  # Convert to PIL Image
        if self.transform:
            image = self.transform(image)
        return image, label

# Create data loaders
batch_size = 32
dataloaders = {
    'train': DataLoader(CervicalCancerDataset(train_df, transform), batch_size=batch_size, shuffle=True),
    'valid': DataLoader(CervicalCancerDataset(valid_df, transform), batch_size=batch_size, shuffle=False),
    'test': DataLoader(CervicalCancerDataset(test_df, transform), batch_size=batch_size, shuffle=False)
}

# Define model
class CervicalCancerModel(nn.Module):
    def __init__(self):
        super(CervicalCancerModel, self).__init__()
        self.base_model = models.resnet101(pretrained=True)
        for param in self.base_model.parameters():
            param.requires_grad = False
        self.base_model.fc = nn.Sequential(
            nn.Linear(self.base_model.fc.in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, len(cervix_categories))
        )
    
    def forward(self, x):
        return self.base_model(x)

# Initialize model, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CervicalCancerModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adamax(model.parameters(), lr=0.001)

c:\Anaconda\envs\py310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Anaconda\envs\py310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
# Define model
class CervicalCancerModel(nn.Module):
    def __init__(self):
        super(CervicalCancerModel, self).__init__()
        self.base_model = models.mobilenet_v3_large(pretrained=True)
        
        # Freeze the parameters of the base model
        for param in self.base_model.parameters():
            param.requires_grad = False
        
        # The number of output features from the last layer of the base model
        num_features = 960  # This should match the output of the last layer of MobileNetV3 large
        
        # Redefine the classifier
        self.base_model.classifier = nn.Sequential(
            nn.Linear(num_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, len(cervix_categories))
        )

    def forward(self, x):
        return self.base_model(x)


# Assuming cervix_categories is defined somewhere in your code
# If not, you'll need to define it like:
# cervix_categories = ['category1', 'category2', ...] 

# Initialize model, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CervicalCancerModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adamax(model.parameters(), lr=0.001)

c:\Anaconda\envs\py310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Anaconda\envs\py310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:

# Training function
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            correct = 0
            total = 0
            
            loop = tqdm(dataloaders[phase], desc=f'{phase.upper()} Epoch {epoch+1}/{num_epochs}', leave=True)
            for inputs, labels in loop:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * inputs.size(0)
                correct += torch.sum(preds == labels.data)
                total += labels.size(0)
                
                loop.set_postfix(loss=running_loss/total, accuracy=correct.double()/total)
            
            epoch_loss = running_loss / total
            epoch_acc = correct.double() / total
            print(f'{phase.upper()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

train_model(model, dataloaders, criterion, optimizer)

# Save model
torch.save(model.state_dict(), "cervical_cancer_model.pth")


Epoch 1/10


TRAIN Epoch 1/10: 100%|██████████| 547/547 [04:50<00:00,  1.88it/s, accuracy=tensor(0.8654, device='cuda:0', dtype=torch.float64), loss=0.409]


TRAIN Loss: 0.4091 Acc: 0.8654


VALID Epoch 1/10: 100%|██████████| 118/118 [01:17<00:00,  1.52it/s, accuracy=tensor(0.9411, device='cuda:0', dtype=torch.float64), loss=0.181]


VALID Loss: 0.1814 Acc: 0.9411

Epoch 2/10


...

VALID Loss: 0.0169 Acc: 0.9963

Epoch 10/10


TRAIN Epoch 10/10: 100%|██████████| 547/547 [02:15<00:00,  4.04it/s, accuracy=tensor(0.9887, device='cuda:0', dtype=torch.float64), loss=0.038] 


TRAIN Loss: 0.0380 Acc: 0.9887


VALID Epoch 10/10: 100%|██████████| 118/118 [00:29<00:00,  4.05it/s, accuracy=tensor(0.9955, device='cuda:0', dtype=torch.float64), loss=0.0172]


VALID Loss: 0.0172 Acc: 0.9955


In [ ]:
# Define model

# Define model
class CervicalCancerModel(nn.Module):
    def __init__(self):
        super(CervicalCancerModel, self).__init__()
        # Use a specific pre-trained Vision Transformer
        self.base_model = models.vit_b_16(pretrained=True)
        
        # Freeze base model parameters
        for param in self.base_model.parameters():
            param.requires_grad = False
        
        # Get the correct number of input features for heads
        num_features = 768  # 768 for vit_b_16
        
        # Replace heads with a new trainable classifier
        self.base_model.heads = nn.Sequential(
            nn.Linear(num_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, len(cervix_categories))
        )
    
    def forward(self, x):
        return self.base_model(x)

# Initialize model, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CervicalCancerModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adamax(model.parameters(), lr=0.001)

In [23]:
# Training function
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            correct = 0
            total = 0
            
            loop = tqdm(dataloaders[phase], desc=f'{phase.upper()} Epoch {epoch+1}/{num_epochs}', leave=True)
            for inputs, labels in loop:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * inputs.size(0)
                correct += torch.sum(preds == labels.data)
                total += labels.size(0)
                
                loop.set_postfix(loss=running_loss/total, accuracy=correct.double()/total)
            
            epoch_loss = running_loss / total
            epoch_acc = correct.double() / total
            print(f'{phase.upper()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

train_model(model, dataloaders, criterion, optimizer)

# Save model
torch.save(model.state_dict(), "cervical_cancer_model_Vision_Transformer.pth")


Epoch 1/10


TRAIN Epoch 1/10:   0%|          | 0/547 [00:00<?, ?it/s]

TRAIN Epoch 1/10: 100%|██████████| 547/547 [10:34<00:00,  1.16s/it, accuracy=tensor(0.9151, device='cuda:0', dtype=torch.float64), loss=0.262]


TRAIN Loss: 0.2624 Acc: 0.9151


VALID Epoch 1/10: 100%|██████████| 118/118 [01:58<00:00,  1.01s/it, accuracy=tensor(0.9667, device='cuda:0', dtype=torch.float64), loss=0.114]


...

VALID Loss: 0.0111 Acc: 0.9976

Epoch 10/10


TRAIN Epoch 10/10: 100%|██████████| 547/547 [14:33<00:00,  1.60s/it, accuracy=tensor(0.9994, device='cuda:0', dtype=torch.float64), loss=0.00537]  


TRAIN Loss: 0.0054 Acc: 0.9994


VALID Epoch 10/10: 100%|██████████| 118/118 [01:56<00:00,  1.01it/s, accuracy=tensor(0.9979, device='cuda:0', dtype=torch.float64), loss=0.0096] 


VALID Loss: 0.0096 Acc: 0.9979


In [ ]:
# Define model
class CervicalCancerModel(nn.Module):
    def __init__(self):
        super(CervicalCancerModel, self).__init__()
        self.base_model = models.efficientnet_v2_s(pretrained=True)
        # Freeze the base model parameters
        for param in self.base_model.parameters():
            param.requires_grad = False
        # Get the number of input features from the classifier's linear layer
        num_features = self.base_model.classifier[1].in_features
        # Replace the classifier with a custom sequential module
        self.base_model.classifier = nn.Sequential(
            nn.Linear(num_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, len(cervix_categories))
        )
    
    def forward(self, x):
        return self.base_model(x)

# Initialize model, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CervicalCancerModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adamax(model.parameters(), lr=0.001)

c:\Anaconda\envs\py310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Anaconda\envs\py310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_M_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_M_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_v2_m-dc08266a.pth" to C:\Users\Nguyen Ngo/.cache\torch\hub\checkpoints\efficientnet_v2_m-dc08266a.pth
100%|██████████| 208M/208M [00:03<00:00, 68.0MB/s] 


In [28]:
# Training function
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            correct = 0
            total = 0
            
            loop = tqdm(dataloaders[phase], desc=f'{phase.upper()} Epoch {epoch+1}/{num_epochs}', leave=True)
            for inputs, labels in loop:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * inputs.size(0)
                correct += torch.sum(preds == labels.data)
                total += labels.size(0)
                
                loop.set_postfix(loss=running_loss/total, accuracy=correct.double()/total)
            
            epoch_loss = running_loss / total
            epoch_acc = correct.double() / total
            print(f'{phase.upper()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

train_model(model, dataloaders, criterion, optimizer)

# Save model
torch.save(model.state_dict(), "cervical_cancer_model_Efficient_NetV2.pth")


Epoch 1/10


TRAIN Epoch 1/10: 100%|██████████| 547/547 [04:07<00:00,  2.21it/s, accuracy=tensor(0.7571, device='cuda:0', dtype=torch.float64), loss=0.701]


TRAIN Loss: 0.7009 Acc: 0.7571


VALID Epoch 1/10: 100%|██████████| 118/118 [00:51<00:00,  2.30it/s, accuracy=tensor(0.8448, device='cuda:0', dtype=torch.float64), loss=0.445]


VALID Loss: 0.4449 Acc: 0.8448

Epoch 2/10


...

VALID Loss: 0.1822 Acc: 0.9400

Epoch 10/10


TRAIN Epoch 10/10: 100%|██████████| 547/547 [04:18<00:00,  2.11it/s, accuracy=tensor(0.9065, device='cuda:0', dtype=torch.float64), loss=0.265]


TRAIN Loss: 0.2654 Acc: 0.9065


VALID Epoch 10/10: 100%|██████████| 118/118 [00:56<00:00,  2.10it/s, accuracy=tensor(0.9472, device='cuda:0', dtype=torch.float64), loss=0.158]


VALID Loss: 0.1578 Acc: 0.9472


In [8]:
# Define model
class CervicalCancerModel(nn.Module):
    def __init__(self):
        super(CervicalCancerModel, self).__init__()
        self.base_model = models.densenet201(pretrained=True)
        # Freeze the base model parameters
        for param in self.base_model.parameters():
            param.requires_grad = False
        # Get the number of input features from the classifier's linear layer
        num_features = self.base_model.classifier.in_features  # Fixed line
        # Replace the classifier with a custom sequential module
        self.base_model.classifier = nn.Sequential(
            nn.Linear(num_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, len(cervix_categories))
        )
    
    def forward(self, x):
        return self.base_model(x)

# Initialize model, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CervicalCancerModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adamax(model.parameters(), lr=0.001)

In [9]:
# Training function
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            correct = 0
            total = 0
            
            loop = tqdm(dataloaders[phase], desc=f'{phase.upper()} Epoch {epoch+1}/{num_epochs}', leave=True)
            for inputs, labels in loop:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * inputs.size(0)
                correct += torch.sum(preds == labels.data)
                total += labels.size(0)
                
                loop.set_postfix(loss=running_loss/total, accuracy=correct.double()/total)
            
            epoch_loss = running_loss / total
            epoch_acc = correct.double() / total
            print(f'{phase.upper()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

train_model(model, dataloaders, criterion, optimizer)

# Save model
torch.save(model.state_dict(), "cervical_cancer_model_Densnet201.pth")


Epoch 1/10


TRAIN Epoch 1/10: 100%|██████████| 547/547 [04:12<00:00,  2.17it/s, accuracy=tensor(0.8833, device='cuda:0', dtype=torch.float64), loss=0.36] 


TRAIN Loss: 0.3601 Acc: 0.8833


VALID Epoch 1/10: 100%|██████████| 118/118 [00:52<00:00,  2.23it/s, accuracy=tensor(0.9661, device='cuda:0', dtype=torch.float64), loss=0.129]


VALID Loss: 0.1292 Acc: 0.9661

Epoch 2/10


...

VALID Loss: 0.0161 Acc: 0.9949

Epoch 10/10


TRAIN Epoch 10/10: 100%|██████████| 547/547 [03:49<00:00,  2.38it/s, accuracy=tensor(0.9898, device='cuda:0', dtype=torch.float64), loss=0.033] 


TRAIN Loss: 0.0330 Acc: 0.9898


VALID Epoch 10/10: 100%|██████████| 118/118 [00:48<00:00,  2.41it/s, accuracy=tensor(0.9976, device='cuda:0', dtype=torch.float64), loss=0.0118]


VALID Loss: 0.0118 Acc: 0.9976
